# Importing Dependencies

In [5]:
import pandas
import numpy

# Importing_data

In [6]:
df=pandas.read_csv('/content/constitution_qa.csv')

In [7]:
df

,question,answer
0,What is India according to the Union and its T...,"India, that is Bharat, shall be a Union of Sta..."
1,"How is India, that is Bharat, defined in terms...","India, that is Bharat, is defined as a Union o..."
2,What does the territory of India comprise of?,The territory of India shall comprise the terr...
3,"What does the territory of a country, such as ...",The territory of a country like India comprise...
4,Who has the authority to admit or establish ne...,"Parliament may by law admit into the Union, or..."
...,...,...
4077,Which schedule includes small scale industries...,The Eleventh Schedule includes these industries.
4078,What welfare programs are included in the Elev...,"Family welfare, Women and child development, S..."
4079,Which Schedule includes programs such as famil...,These programs are included in the Eleventh Sc...
4080,What does the twelfth schedule of Article 243W...,"Urban planning including town planning, regula..."


In [8]:
from datasets import Dataset,DatasetDict

In [9]:
dataset = Dataset.from_pandas(df)
dataset = dataset.shuffle(seed=0)
dataset = dataset.train_test_split(test_size=0.1)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 3673
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 409
    })
})

In [11]:
dataset['test'][2]

{'question': "Where is the administration of an autonomous district vested if it's not under the Regional Council?",
 'answer': 'In the District Council for such district.'}

# Writing a formatting Function which will guide the model

In [12]:
def formatting_func(example):
    text = f"""
            Given the question, you need to generate an answer '{example['answer']}'.
            ### question: {example['question']}\n ### {example['answer']}

            """
    return text

In [13]:
print(formatting_func(dataset['train'][0]))


            Given the question, you need to generate an answer 'The duties of excise are applied on goods manufactured or produced in the state, specifically on alcoholic liquors for human consumption, opium, Indian hemp and other narcotic drugs and narcotics, but not including medicinal and toilet preparations containing alcohol or any substance included in sub-paragraph (b) of this entry.'.
            ### question: On what goods are the duties of excise applied in the state, specifically in terms of alcoholic beverages, opium, Indian hemp, and other narcotic drugs and narcotics?
 ### The duties of excise are applied on goods manufactured or produced in the state, specifically on alcoholic liquors for human consumption, opium, Indian hemp and other narcotic drugs and narcotics, but not including medicinal and toilet preparations containing alcohol or any substance included in sub-paragraph (b) of this entry.

            


## Load Base Model and Tokenize

In [11]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 838.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

# Loading microsoft/phi-2 in 8bit format

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True,
                                             torch_dtype=torch.float16, load_in_8bit=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_size='left',
    add_eos_token=True,
    add_bos_token=True,
    use_fast=False
)

tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [3]:
max_length = 400

def tokenize(prompt):
  result = tokenizer(
      formatting_func(prompt),
      truncation = True,
      max_length=max_length,
      padding = "max_length",
  )

  result['labels'] = result['input_ids'].copy()

  return result

In [14]:
print(tokenize(dataset['train'][0]))

{'input_ids': [50256, 198, 50276, 15056, 262, 1808, 11, 345, 761, 284, 7716, 281, 3280, 705, 464, 10741, 286, 47547, 389, 5625, 319, 7017, 15943, 393, 4635, 287, 262, 1181, 11, 5734, 319, 26016, 14756, 669, 329, 1692, 7327, 11, 40299, 11, 3942, 28246, 290, 584, 16946, 6210, 5010, 290, 36524, 11, 475, 407, 1390, 30381, 290, 16146, 21518, 7268, 5548, 393, 597, 9136, 3017, 287, 850, 12, 20360, 357, 65, 8, 286, 428, 5726, 2637, 13, 198, 50276, 21017, 1808, 25, 1550, 644, 7017, 389, 262, 10741, 286, 47547, 5625, 287, 262, 1181, 11, 5734, 287, 2846, 286, 26016, 24173, 11, 40299, 11, 3942, 28246, 11, 290, 584, 16946, 6210, 5010, 290, 36524, 30, 198, 44386, 383, 10741, 286, 47547, 389, 5625, 319, 7017, 15943, 393, 4635, 287, 262, 1181, 11, 5734, 319, 26016, 14756, 669, 329, 1692, 7327, 11, 40299, 11, 3942, 28246, 290, 584, 16946, 6210, 5010, 290, 36524, 11, 475, 407, 1390, 30381, 290, 16146, 21518, 7268, 5548, 393, 597, 9136, 3017, 287, 850, 12, 20360, 357, 65, 8, 286, 428, 5726, 13, 628, 5027

In [15]:
dataset = dataset.map(tokenize)

Map:   0%|          | 0/3673 [00:00<?, ? examples/s]

Map:   0%|          | 0/409 [00:00<?, ? examples/s]

In [16]:
from peft import LoraConfig, get_peft_model

target_modules = ["Wqkv", "fc1", "fc2"]

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules = target_modules,
    bias = "none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)



In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [18]:
print_trainable_parameters(model)

trainable params: 26214400 || all params: 2805898240 || trainable%: 0.9342605382581515


## Model Training

In [19]:
from accelerate import Accelerator

accelerator = Accelerator(gradient_accumulation_steps=1)

model = accelerator.prepare_model(model)

In [20]:
import tqdm
tqdm.tqdm.pandas()

In [22]:
# Trainer, Training Arguments, DataCollator

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datetime import datetime

project = "phi2-finetune"
run_name = 'train-dir'
output_dir = "./" + run_name

args=TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=500,
        learning_rate=2.5e-5,
        optim="paged_adamw_8bit",
        logging_steps=25,
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=25,
        eval_strategy="steps",
        eval_steps=250,
        do_eval=True,
        report_to='none'
    )

trainer = Trainer(
    model=model,
    args = args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
250,1.090700,1.100926
500,1.109000,1.044528


TrainOutput(global_step=500, training_loss=1.3108665809631348, metrics={'train_runtime': 644.7963, 'train_samples_per_second': 1.551, 'train_steps_per_second': 0.775, 'total_flos': 6419582976000000.0, 'train_loss': 1.3108665809631348, 'epoch': 0.2721829069134458})

In [23]:
trainer.save_model("phi2-finetuned")
tokenizer.save_pretrained("phi2-finetuned")

('phi2-finetuned/tokenizer_config.json',
 'phi2-finetuned/special_tokens_map.json',
 'phi2-finetuned/vocab.json',
 'phi2-finetuned/merges.txt',
 'phi2-finetuned/added_tokens.json')

# will  answer user's query on indian constitution

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("phi2-finetuned")
tokenizer = AutoTokenizer.from_pretrained("phi2-finetuned",use_fast=False)
model.to('cuda')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): lora.Linear(
            (base_layer): Linear(in_features=2560, out_features=10240, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=

In [2]:
from transformers import pipeline

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

user_input = "What are the ways in which a new State can be formed?"
output = text_generator(user_input, max_new_tokens=100, do_sample=True, temperature=0.7)

print(output[0]['generated_text'])

Device set to use cuda:0


What are the ways in which a new State can be formed?
By: Mr. K. S. Murthy
In the Union List, paragraph (b), clause (2), the word 'States' includes the Union territory, which was excluded from the Seventh Schedule, subject to the provisions of this article.
The Parliament shall by law provide for the formation of States, including the Union territory, and may by law alter the boundaries of any State or Union territory.
The Parliament may by law in any case for the formation of States; or for
